In [0]:
dbutils.widgets.text("p_data_date", "")
v_data_date = dbutils.widgets.get("p_data_date")

In [0]:
formular1_key = dbutils.secrets.get(scope='fomular1-scope', key='formuladl1-key-account')


In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", formular1_key)

In [0]:
from pyspark.sql.functions import col, lit, to_timestamp, concat, current_timestamp, to_date, substring, regexp_extract, to_timestamp
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType, TimestampType



In [0]:
qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("q1", StringType(), True),
                                      StructField("q2", StringType(), True),
                                      StructField("q3", StringType(), True)
                                     ])


In [0]:
qualifying_df = spark.read.schema(qualifying_schema).option("multiLine", True).json(f"abfss://raw@fomular1dl1.dfs.core.windows.net/{v_data_date}/qualifying")

In [0]:
qualifying_df.show()

In [0]:
final_df = qualifying_df.withColumnRenamed("raceId", "race_id") \
                        .withColumnRenamed("qualifyId", "qualify_id") \
                       .withColumnRenamed("driverId", "driver_id") \
                       .withColumnRenamed("constructorId", "constructor_id") \
                       .withColumn("ingestion_date", current_timestamp())


In [0]:
final_df.show()

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.qualifying")